**FEATURE ENGINEERING** <br>
создание копии <br>
удление столбцов <br>
математические операции со столбцами <br>
DATTIME интервалы <br>
nunique <br>
функции для фильтрации<br>
CATEGORY


In [4]:
import pandas as pd

СОЗДАНИЕ КОПИИ ТАБЛИЦЫ

In [5]:
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
#melb_data.head()

melb_df = melb_data.copy()#создание копии
#melb_df.head()


УДАЛЕНИЕ СТОЛБЦОВ

In [2]:
#удалили столбцы index и Coordinates

#melb_df = melb_df.drop(['index', 'Coordinates'], axis=1) #Удалим столбцы index и Coordinates
#melb_df.head()


#melb_df.drop(['index','Coordinates'],axis=1,inplace=True) #альтернативный вариант
#melb_df.head()

МАТЕМАТИЧЕСКИЕ ОПЕРАЦИИ СО СТОЛБЦАМИ

In [47]:
#создали признак средней площади одной комнаты MeanRoomsSquare и ввели коэффициент соотношения
# площади здания к площади участка — AreaRatio

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
#display(total_rooms)

melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms #средней площади одной комнаты для каждого объекта
#display(melb_df['MeanRoomsSquare'])

diff_area = melb_df['BuildingArea'] - melb_df['Landsize'] #AreaRatio, коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). 
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])

ПРИЗНАКИ ДАТЫ И ВРЕМЕНИ <br>
ФОРМАТ DATETIME <br>
РАБОТА С ИНТЕРВАЛАМИ

In [48]:
#преобразовали признак даты продажи Date в формат datetime

#display(melb_df['Date'])melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# #Преобразуем столбец Date в формат datetime

#display(melb_df['Date'])

In [49]:
years_sold = melb_df['Date'].dt.year #«достать» год ,РЕЗУЛЬТАТ В ПЕРЕМЕННОЙ
#print(years_sold)
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
#print('Mode year sold:', years_sold.mode()[0])

In [50]:
# создали на его основе следующие столбцы: номер месяц продажи (MonthSale),
# номер дня недели продажи (WeekdaySale), «признак-мигалку» выходного дня (Weekend) ?

melb_df['MonthSale'] = melb_df['Date'].dt.month #РЕЗУЛЬТАТ В ТАБЛИЦУ
melb_df['MonthSale'].value_counts(normalize=True)

#ИНТЕРВАЛЫ
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') #!!!!!!!!!!!
#display(delta_days)
#display(delta_days.dt.days) # количество дней из формата интервала в формат целого числа дней

In [52]:
#заменили признак года постройки здания YearBuilt на его возраст AgeBuilding

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])
melb_df = melb_df.drop('YearBuilt', axis=1)

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

Создание и преобразование столбцов с помощью функций

In [68]:
#print(melb_df['Address'].nunique())#считаем уникальные значения в столбце адрес

#извлекли из признака адреса объекта Address новый признак
# подтипа улицы StreetType и удалили столбец с адресом

#print(melb_df['Address'].loc[177])
#print(melb_df['Address'].loc[1812])
#print(melb_df['Address'].loc[9001])


# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type


street_types = melb_df['Address'].apply(get_street_type)#применили к столбцу Эдресс нашу фенкцию к каждой ячейке
#display(street_types)


#print(street_types.nunique())

#display(street_types.value_counts())

#применим очень распространённый метод уменьшения количества уникальных категорий —
# выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' 

popular_stypes =street_types.value_counts().nlargest(10).index #частота уник значений .самых больших n=10 .их названия index
print(popular_stypes)

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other') # есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x,
#в противном случае она будет возвращать строку 'other'
#display(melb_df['StreetType'])

print(melb_df['StreetType'].nunique())

melb_df = melb_df.drop('Address', axis=1)


Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')
11


In [6]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Rooms,9,int64
4,Bathroom,9,int64
5,Car,11,int64
6,Bedroom,12,int64
7,CouncilArea,33,object
8,Date,58,object
9,YearBuilt,144,int64


ТИП ДАННЫХ CATEGORY

In [10]:
#display(melb_df.info()) #ПАМЯТЬ 2.4

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

# объем памяти 1.9

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

None

In [14]:
#print(melb_df['Regionname'].cat.categories) #уникальных категорий в столбце Regionname

#display(melb_df['Regionname'].cat.codes) #каким образом столбец кодируется в виде чисел в памяти компьютера

#Переименуем категории признака типа постройки
#Type — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
#display(melb_df['Type'])

#тип данных category хранит только категории, которые были объявлены при его инициализации.
# При встрече с новой, неизвестной ранее категорией, этот тип превратит
# её в пустое значение, так как он просто не знает о существовании этой категории.
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']